## Ex 1

## Ex 2

In [1]:
import numpy as np
from operator import itemgetter, not_
from bisect import bisect_left
from time import time

In [2]:
bound = 10
m = 100

rnd = lambda: np.random.uniform(-bound, bound)
h = lambda a, x: (a <= x and x <= a + 1) or (a + 2 <= x and x <= a + 4)
loss = lambda h, S: sum([h(x) != y for x, y in S])

def test_case(S):
    learned_a = learn_a(S)
    h_erm = lambda x: h(learned_a, x)
    return loss(h_erm, S)

debug = False
def debug_print(*args, **kwargs):
    if debug: print(*args, **kwargs)
    
longest_interval = 5

def learn_a(S):
    
    S.sort(key=itemgetter(0))
    
    Sx = list(map(itemgetter(0), S))
    Sy = list(map(itemgetter(1), S))
    
    x_min, x_max = min(Sx), max(Sx)
    
    if all(Sy):
        debug_print(f"All positive case. a = {x_min}")
        return x_min
    
    if all(map(not_, Sy)):
        debug_print(f"All negative case. a = {x_max + 1}")
        return x_max + 1
        
    a_best, loss_best = np.inf, np.inf
    
    a_curr = x_min - 6 - 1
    iteration = 0
    
    h_curr = lambda x: h(a_curr, x)
    loss_curr = loss(h_curr, S)
    
    will_change_i = 0
    old_classification = h_curr(Sx[will_change_i]) == Sy[will_change_i]
    
    while a_curr <= x_max:

        h_curr = lambda x: h(a_curr, x)
        new_classification = h_curr(Sx[will_change_i]) == Sy[will_change_i]
        
        if old_classification and new_classification:
            pass
        elif old_classification:
            loss_curr += 1
        elif new_classification:
            loss_curr -= 1
            
        debug_print(f"iteration {iteration}: a = {a_curr} loss = {loss_curr}")
            
#         correct_loss = loss(h_curr, S)
#         assert loss_curr == correct_loss
        
        if loss_curr < loss_best:
            loss_best = loss_curr
            a_best = a_curr
            
            debug_print(f"new best: a = {a_curr} loss = {loss_curr}")
        
        margins = [a_curr, a_curr + 1, a_curr + 2, a_curr + 4]
        margin_dist_min = np.inf
        will_change_i = np.inf
        
        for margin in margins:
            i = bisect_left(Sx, margin)
            if i >= len(Sx) or Sx[i] < margin: continue
            
            if Sx[i] - margin < margin_dist_min:
                will_change_i = i
                margin_dist_min = Sx[i] - margin
                    
        assert margin_dist_min >= 0 and margin_dist_min != np.inf and will_change_i != np.inf
        
        old_classification = h_curr(Sx[will_change_i]) == Sy[will_change_i]
        a_curr += margin_dist_min + 1e-6
        
        iteration += 1
    
    debug_print(f"solution: a = {a_best} loss = {loss_best}")
    
    return a_best

In [3]:
# Trivial test cases
# All true
assert test_case([(1, True), (2, True), (4, True)]) == 0
# All false
assert test_case([(5, False), (6, False)]) == 0
# All in first
assert test_case([(1, True), (1.5, True), (2.1, False)]) == 0
# All in second
assert test_case([(1.9, False), (2, True), (3.5, True), (4.01, False)]) == 0

# Random test case
n = 100
start_time = time()
for i in range(n):
#     print(f"Random test case {i+1}/{n}")
    
    true_a = rnd() / 2
    f = lambda x: h(true_a, x)

    Sx = [rnd() for _ in range(m)]
    S = [(x, f(x) if np.random.random() < 0.8 else not f(x)) for x in Sx]

    assert any([y for x, y in S])
    assert any([not y for x, y in S])

    test_case_loss = test_case(S)
#     print(test_case_loss, loss(f, S))
    assert np.isclose(test_case_loss, loss(f, S)) or test_case_loss < loss(f, S)
    
time() - start_time

0.3243281841278076

## Ex 4

In [2]:
from itertools import product
import numpy as np

def generate_nfs(n, k):
    clauses = list(product([-1, 0, +1], repeat=n))
    nfs = list(product(clauses, repeat=k))
    
    return nfs
    
def dnf(x, form):
    for clause in form:
        conj_result = True
        for i, literal in enumerate(clause):
            if literal == 0: 
                continue
            
            conj_result = conj_result and (x[i] if literal > 0 else not x[i])
        
        if conj_result:
            return True
        
        
    return False
    
def generate_x(n):
    return [bool(np.random.binomial(1, 0.5)) for _ in range(n)]

def compute_error(S, h, true_nf):
    return sum([h(x) != dnf(x, true_nf) for x in S]) / len(S)

In [3]:
def weak_h(x, i, j):
    x_i = x[i - 1] if i > 0 else not x[-i - 1]
    x_j = x[j - 1] if j > 0 else not x[-j - 1]
    return x_i or x_j

def learn_h(S, true_nf):
    best_h = lambda x: 0
    best_h_type = None
    best_error = compute_error(S, best_h, true_nf) 

    literals = list(range(-N, 0)) + list(range(1, N + 1))
    for i, j in product(literals, repeat=2):
        if i == -j: continue

        h_curr = lambda x: weak_h(x, i, j)

        error_curr = compute_error(S, h_curr, true_nf)

        if error_curr < best_error and error_curr < 0.5:
            best_error = error_curr
            best_h = h_curr
            best_h_type = (i, j)
            
    return best_error, best_h_type, best_h
        
learn_h(S, true_nf)

NameError: name 'true_nf' is not defined

In [6]:
N = 3
K = 2
M = 6
RETRIES = 2

constant_h = lambda x: 0

vals = []
constant_vals = []

nfs = generate_nfs(N, K)

for true_nf in nfs:
    for _ in range(RETRIES):
        S = [generate_x(N) for i in range(M)]
        best_error, best_h_type, best_h = learn_h(S, true_nf)
#         print(best_error, best_h_type)
        
        large_S = [generate_x(N) for i in range(100*M)]
        e = compute_error(large_S, best_h, true_nf)
        constant_e = compute_error(large_S, constant_h, true_nf)
        
        if constant_e < e:
            print("warning!")
        
        vals.append(e)
        constant_vals.append(constant_e)
        
        
sorted(vals[:5], reverse=True), sorted(constant_vals[:5], reverse=True)

warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
w

([0.62,
  0.4716666666666667,
  0.23333333333333334,
  0.23333333333333334,
  0.13833333333333334],
 [0.23333333333333334,
  0.23333333333333334,
  0.22166666666666668,
  0.13833333333333334,
  0.115])

In [35]:
import numpy as np

def weak_learn(S, y):
    d = S.shape[1]
    best_i, best_j, best_error = None, None, float('inf')
    # Iterate over all pairs of literals
    for i in range(d):
        for j in range(i+1, d):
            # Construct a hypothesis
            h = np.logical_or(S[:, i], S[:, j])
            error = np.mean(h != y)
            # Update best literals if error is lower
            if error < best_error:
                best_i, best_j, best_error = i, j, error

    # Error of the constant classifier
    constant_error = min(np.mean(y), 1 - np.mean(y))

    if best_error < constant_error:
        # If best pair of literals has smaller error than the constant classifier
        return np.logical_or(S[:, best_i], S[:, best_j])
    else:
        # Otherwise, return the majority class
        majority_class = np.argmax(np.bincount(y))
        return np.full(S.shape[0], majority_class)


def f(random_array):
    return (random_array[:, 0] & random_array[:, 2] & random_array[:, 4]) | (random_array[:, 1] & (~random_array[:, 2]) & random_array[:, 5])
    
# Function to generate a random dataset
def generate_dataset(n, d):
    S = np.random.randint(2, size=(n, d))
    y = f(S)
    return S, y

# Set random seed for reproducibility
# np.random.seed(42)

# Initialize error counters
learner_error = 0
baseline_error = 0

# Number of trials
trials = 1000

# Test the weak learner over many random datasets
for _ in range(trials):
    # Generate a random dataset
    S, y = generate_dataset(100, 10)
    
    # Get the hypothesis from the weak learner
    h = weak_learn(S, y)
    
    # Compute the error rate of the weak learner
    learner_error += np.mean(h != y)
    
    # Compute the baseline error rate (assuming random guessing)
    baseline_error += 0.5

# Average the error rates
learner_error /= trials
baseline_error /= trials

print(f'Error rate of weak learner: {learner_error}')
print(f'Baseline error rate: {baseline_error}')

Error rate of weak learner: 0.40616
Baseline error rate: 0.5


In [21]:
import numpy as np
import pprint

def weak_learn(S, y):
    d = S.shape[1]
    best_i, best_j, best_error = None, None, float('inf')
    # Iterate over all pairs of literals
    for i in range(d):
        for j in range(i+1, d):
            # Construct a hypothesis
            h = np.logical_or(S[:, i], S[:, j])
            error = np.mean(h != y)
            # Update best literals if error is lower
            if error < best_error:
                best_i, best_j, best_error = i, j, error

    # Error of the constant classifier
    constant_error = min(np.mean(y), 1 - np.mean(y))

    if best_error < constant_error:
        # If best pair of literals has smaller error than the constant classifier
        return np.logical_or(S[:, best_i], S[:, best_j])
    else:
        # Otherwise, return the majority class
        majority_class = np.argmax(np.bincount(y))
        return np.full(S.shape[0], majority_class)

def f(random_array):
    return (random_array[:, 0] & random_array[:, 2] & random_array[:, 4]) | (random_array[:, 1] & (~random_array[:, 2]) & random_array[:, 5])
    
# Function to generate a random dataset
def generate_dataset(n, d):
    S = np.random.randint(2, size=(n, d))
    y = f(S)
    return S, y


# Set random seed for reproducibility
# np.random.seed(42)

# Initialize error counters
learner_error = 0
baseline_error = 0

# Number of trials
trials = 1000

# Positive class ratio
pos_ratio = 0.50

# Test the weak learner over many random imbalanced datasets
for _ in range(trials):
    # Generate a random imbalanced dataset
    S, y = generate_dataset(5, 7)
    
    # Get the hypothesis from the weak learner
    h = weak_learn(S, y)
    
    # Compute the error rate of the weak learner
    for a, b in zip(h, y):
        if a != b and not b:
            print("Umm?")
            print(S)
            
        
    learner_error += np.sum(h != y)
    
    # Compute the baseline error rate (assuming random guessing)
    baseline_error += min(np.sum(y), len(y) - np.sum(y))

# Average the error rates
# learner_error /= trials
# baseline_error /= trials

print(f'Error rate of weak learner: {learner_error}')
print(f'Baseline error rate: {baseline_error}')


Umm?
[[1 1 1 1 1 1 0]
 [0 0 0 1 1 0 1]
 [1 0 0 1 1 1 1]
 [1 0 1 0 1 1 1]
 [0 1 0 1 1 0 0]]
Umm?
[[1 1 1 1 1 1 1]
 [0 1 0 1 0 0 0]
 [1 1 0 0 0 0 1]
 [1 1 1 1 1 0 1]
 [0 0 1 0 0 1 1]]
Umm?
[[1 1 1 0 1 0 1]
 [1 0 1 0 0 1 1]
 [0 0 1 1 0 0 1]
 [0 1 1 1 0 0 0]
 [1 1 1 1 1 0 0]]
Umm?
[[1 1 1 1 0 0 1]
 [1 0 1 0 1 1 1]
 [1 0 1 1 1 1 1]
 [0 0 0 1 1 0 1]
 [0 1 1 0 1 0 1]]
Umm?
[[1 1 0 1 1 1 1]
 [0 0 0 1 1 1 0]
 [0 1 1 0 0 1 1]
 [1 1 1 1 1 1 0]
 [0 1 0 0 0 1 1]]
Umm?
[[1 0 1 0 1 1 1]
 [0 1 0 1 1 0 0]
 [0 0 0 1 0 1 1]
 [1 0 0 0 1 1 1]
 [1 1 0 1 0 1 1]]
Umm?
[[0 0 0 1 1 0 0]
 [0 1 0 1 1 0 0]
 [1 1 1 1 1 0 1]
 [1 1 1 0 1 1 1]
 [1 0 1 1 0 0 1]]
Umm?
[[0 1 1 1 0 0 1]
 [0 1 0 1 0 1 1]
 [0 1 0 1 1 1 1]
 [0 0 1 1 0 0 0]
 [0 0 1 0 0 1 0]]
Umm?
[[1 1 0 1 0 0 1]
 [1 1 1 0 1 0 0]
 [1 0 1 0 1 1 0]
 [0 1 1 0 1 1 1]
 [0 0 0 1 1 0 1]]
Umm?
[[1 1 1 1 0 1 0]
 [1 0 1 1 1 1 0]
 [1 1 1 1 1 0 0]
 [1 1 1 1 0 1 0]
 [0 1 0 1 1 0 1]]
Umm?
[[0 1 0 1 1 0 0]
 [0 1 0 0 1 1 0]
 [1 1 1 0 0 0 1]
 [0 1 0 0 0 1 0]
 [1 0 1 1 1 1 0]]

Umm?
[[1 1 0 0 1 1 0]
 [1 0 0 0 1 1 1]
 [1 1 1 1 0 1 0]
 [1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1]]
Umm?
[[1 1 1 1 1 0 0]
 [1 0 0 1 0 1 1]
 [1 0 0 1 1 0 1]
 [1 0 1 1 0 0 0]
 [1 1 1 1 1 0 1]]
Umm?
[[1 0 1 0 0 0 1]
 [0 1 0 1 0 1 0]
 [1 0 0 1 1 1 1]
 [1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0]]
Umm?
[[1 1 1 0 0 1 1]
 [0 1 1 0 0 1 1]
 [0 1 0 1 0 1 0]
 [1 1 1 0 1 0 1]
 [0 1 0 0 1 0 0]]
Umm?
[[0 1 0 0 0 0 1]
 [1 1 1 0 0 1 0]
 [1 1 1 1 1 1 0]
 [0 1 0 1 1 1 0]
 [0 1 1 1 0 1 0]]
Umm?
[[1 0 0 1 0 0 1]
 [0 0 1 1 0 1 0]
 [1 1 0 1 0 1 0]
 [0 1 1 0 0 1 1]
 [1 1 0 1 0 1 0]]
Umm?
[[0 0 1 0 1 1 1]
 [1 1 1 1 1 1 0]
 [0 0 1 0 1 1 1]
 [0 1 0 0 0 1 1]
 [0 1 1 0 1 1 0]]
Umm?
[[1 1 0 1 1 0 1]
 [1 1 1 0 1 0 1]
 [0 0 0 1 1 0 1]
 [0 0 1 1 1 1 1]
 [1 1 1 1 1 0 1]]
Umm?
[[1 1 1 0 1 1 1]
 [1 0 1 0 0 1 1]
 [0 1 0 1 0 1 0]
 [0 0 0 0 0 1 0]
 [0 1 0 0 1 0 0]]
Umm?
[[1 1 1 0 0 1 1]
 [0 1 0 1 0 0 1]
 [1 1 1 1 1 1 0]
 [0 1 0 0 0 1 1]
 [1 0 1 0 1 1 1]]
Umm?
[[1 0 1 0 1 0 0]
 [0 0 0 0 0 0 1]
 [1 1 0 1 0 1 1]
 [1 0 0 1 1 1 1]
 [1 1 1 1 0 0 1]]